# SAC to MAT Conversion Workflow

This notebook converts SAC files for each event into MATLAB-compatible `.mat` files.

## Setup
1. Load event catalog and station metadata
2. Loop over events and call `sac2mat()` for each
3. Save output to `../data/matfile/YYYY/MM/evXXXXX.mat`

In [ ]:
import os
import sys
import numpy as np
from scipy.io import savemat
from joblib import Parallel, delayed

# Add src to path for imports
sys.path.insert(0, os.path.abspath('../src'))
from sacstuff.sac2mat import sac2mat

## Configuration

In [ ]:
catfile = 'catalog'
stafile = 'station.txt'
sacdir_in = '../data/sac'
evdir = '../data/matfile'
sactype = 'scec'
mode = 'new'

# Create output directory if needed
os.makedirs(evdir, exist_ok=True)

## Load Event Catalog

Read earthquake catalog and build `eqinfo` dict.  
**Note:** The MATLAB code calls `update_eqinfo(catfile, 8)` which is not available.  
Replace this cell with your own catalog loader that populates:
- `eqinfo['id']`: event IDs
- `eqinfo['qyr']`, `eqinfo['qmon']`, `eqinfo['qdy']`: year/month/day
- `eqinfo['qlat']`, `eqinfo['qlon']`, `eqinfo['qtime']`, `eqinfo['mb']`, etc.

In [ ]:
# Load catalog using update_eqinfo
from update_eqinfo import update_eqinfo

# Check if eqinfo.mat already exists
eqinfo_path = os.path.join(evdir, 'eqinfo.mat')

eqinfo = None
# Parse catalog file
# itype: 1=SCEC, 2=HK, 3=EFS, 4=NCSN, 5=JMA, 6=DD, 7=ANSS, 8=XYZ
catalog_itype = 8  # Adjust based on your catalog format

print(f"Parsing catalog: {catfile} (format type {catalog_itype})")
eqinfo = update_eqinfo(catfile, catalog_itype)

# Save for future use
savemat(eqinfo_path, {'eqinfo': eqinfo})
print(f"Saved eqinfo.mat with {len(eqinfo['id'])} events")

# Normalize eqinfo to a plain dict of 1D numpy arrays (matlab structs load as objects)
if not isinstance(eqinfo, dict):
    eqinfo = {field: getattr(eqinfo, field) for field in eqinfo._fieldnames}
eqinfo = {k: np.atleast_1d(np.asarray(v).squeeze()) for k, v in eqinfo.items()}
if 'id' in eqinfo:
    eqinfo['id'] = eqinfo['id'].astype(int)

print(f"Loaded {len(eqinfo['id'])} events from catalog")

## Load Station Metadata

In [ ]:
sta = {
    'stid': [],
    'slat': [],
    'slon': [],
    'selev': [],
}

if os.path.exists(stafile):
    with open(stafile, 'r') as fid:
        for line in fid:
            parts = line.strip().split()
            if len(parts) >= 4:
                sta['stid'].append(parts[0])
                sta['slat'].append(float(parts[1]))
                sta['slon'].append(float(parts[2]))
                sta['selev'].append(float(parts[3]))

# Convert to numpy arrays for consistency with sac2mat
sta['stid'] = np.array(sta['stid'], dtype=str)
sta['slat'] = np.array(sta['slat'])
sta['slon'] = np.array(sta['slon'])
sta['selev'] = np.array(sta['selev'])

n_sta = sta['stid'].size
print(f"Loaded {n_sta} stations")

## Process Events

Loop over events and convert SAC files to MAT format using parallel processing.

In [ ]:
# Ensure trailing slashes
if not sacdir_in.endswith('/'):
    sacdir_in += '/'
if not evdir.endswith('/'):
    evdir += '/'

# Get event IDs and metadata
event_ids = eqinfo['id']
event_years = eqinfo['qyr']
event_months = eqinfo['qmon']
event_days = eqinfo['qdy']

print(f"Processing {event_ids.size} events...")

In [ ]:
def run_sac2mat(evid, year, month, sacdir_in, evdir,eqinfo, sta):
    
    # Output directory organized by year/month
    outevdir = f"{evdir}{year:04d}/{month:02d}"
    os.makedirs(outevdir, exist_ok=True)

    data = sac2mat(
        sacdir_in,
        outevdir,
        evid,
        eqinfo,
        sta,
    )
    if data:
        print(f"  -> Saved ev{evid}.mat with {data.get('numts', 0)} traces")
    else:
        print(f"  -> No SAC data found for event {evid}")

In [ ]:
n_cores = 8

parallel = Parallel(n_jobs=n_cores, backend='loky', verbose=5)

parallel(delayed(run_sac2mat)(
    int(event_ids[ii]),
    int(event_years[ii]),
    int(event_months[ii]),
    sacdir_in,
    evdir,
    eqinfo,
    sta,
) for ii in range(event_ids.size))

print("\nDone!")